In [1]:
library(conflicted)
library(readxl)
library(gtools)
library(plm)
library(tidyverse)
library(modelsummary)
library(huxtable)
library(dplyr)
library(writexl)
library(sjPlot)
library(DescTools)
library(caTools)
library(psych)
library(pcse)
library(stringr)
conflict_prefer("lag", "stats")
conflict_prefer("Mean", "modelsummary")
conflict_prefer("N", "modelsummary")
conflict_prefer("SD", "modelsummary")
conflicted::conflicts_prefer(plm::lag)

# Override default glance.pgmm for summary control
glance.pgmm <- function(x, robust = FALSE, ...) {
  asummary <- summary(x, robust = robust)
  pdim <- attr(asummary, "pdim")
  data.frame(
    N      = pdim$nT$n * (pdim$nT$T - 1),
    Sargan = asummary$sargan$p.value[["chisq"]],
    AR1    = asummary$m1$p.value[1],
    AR2    = asummary$m2$p.value[1]
  )
}
Path.HOME <- path.expand("~")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
[conflicted] Will prefer stats::lag over any other package.
[conflicted] Will prefer modelsummary::Mean over any other package.
[conflicted] Will prefer modelsummary::N over any other package.
[conflicted] Will prefer modelsummary::SD over any other package.
[conflicted] Removing existing preference.
[conflicted] Will prefer plm::lag over any other package.


In [2]:
gm <- tibble::tribble(
  ~raw, ~clean, ~fmt,
  "N", "N", "%d",
  "Sargan", "Sargan", "%.2f",
  "AR1", "AR(1)", "%.2f",
  "AR2", "AR(2)", "%.2f",
  "nobs", "N", "%d",
  "r.squared","R2","%.2f",
  # "time_effect", "时间效应", "%s",
  # "individual_effect", "个体效应", "%s",
  "utest_top", "拐点", "%s",
  "utest_x_max", "上限", "%.2f",
  "utest_x_min", "下限", "%.2f"
)

SIG <- c("*" = .1, "**" = .05, "***" = .01)

names <- c(
    "SDI" = "SDI",
    "SDI_Average" = "SDI_Average",
    "SDI_Equal" = "SDI_Equal",
    
    "lnFDIstock" = "lnFDIstock",
    "lnFDIstock_winsor" = "lnFDIstock_winsor",
    "I(lnFDIstock^2)" = "lnFDIstock2",
    "I(lnFDIstock_winsor^2)" = "lnFDIstock_winsor2", 
    "WGI" = "WGI",
    "property"="property",
    "environ_regu" = "environ_regu",
    "I(lnFDIstock * WGI)"="lnFDIstock*WGI",
    "I(lnFDIstock * property)"="lnFDIstock*property",
    "I(lnFDIstock * environ_regu)"="lnFDIstock*enviregu",
    
    "economic" = "economic",    
    "military" = "military",
    "finance" = "finance",
    "I(lnFDIstock * finance)"="lnFDIstock*finance",
    
    
     "natural" = "natural",
    "urban" = "urban",

    "lag(lnFDIstock)" = "lnFDIstock", 
    "lag(economic)" = "economic",    
    "lag(WGI)" = "WGI",    
    "lag(military)" = "military",
    "lag(urban)" = "urban"
)

utest <- function(model, var1, var2, fieller = FALSE) {
  dof <- df.residual(model)
  beta <- coef(model)
  covar <- vcov(model)
  b1 <- beta[var1]
  b2 <- beta[var2]
  s11 <- covar[var1, var1]
  s22 <- covar[var2, var2]
  s12 <- covar[var1, var2]
  x_min <- min(model$model[var1])
  x_max <- max(model$model[var1])
  xsq <- df[var1]**2
  mode <- "quad"
  t_min <- (b1 + 2 * (b2) * (x_min)) / sqrt(s11 + 4 * (x_min)^2 * s22 + 4 * (x_min) * s12)
  sl_min <- (b1 + 2 * (b2) * (x_min))
  t_max <- (b1 + 2 * (b2) * (x_max)) / sqrt(s11 + 4 * (x_max)^2 * s22 + 4 * (x_max) * s12)
  sl_max <- (b1 + 2 * (b2) * (x_max))
  top <- -(b1) / (2 * b2)
  t_sac <- min(abs(t_min), abs(t_max))
  if (t_min * t_max > 0) {
    t <- NA
    p <- NA
  } else {
    t <- t_sac
    p <- 2 * pt(t_sac, dof, lower.tail = TRUE)
  }
  stars <- ifelse(p < 0.001, "***", ifelse(p < 0.01, "**", ifelse(p <
    0.05, "*", "")))
  breakpoint <- top
  ret <- list(
    statistics = t,
    p.value = p,
    top = top,
    x_min = x_min,
    x_max = x_max
  )
  ret
}
glance_custom.plm <- function(model, ...) {
  ret <- data.frame(
    time_effect = ifelse(
      model$call$effect == "time" | model$call$effect == "twoways",
      "是",
      "否"
    ),
    twoways_effect = ifelse(
      model$call$effect == "twoways",
      "是",
      "否"
    )
  )
  Xs <- (model %>% formula() %>% as.character())[[3]]
  if (str_detect(Xs, "I\\(.+\\^2\\)")) {
    reg.variables <- (formula(model) %>% as.character())[[3]] %>% str_split("\\s*\\+\\s*")
    utest_res <- utest(model, reg.variables[[1]][1], reg.variables[[1]][2])
    stars <- ifelse(utest_res$p.value < 0.001, "***", ifelse(utest_res$p.value <
      0.01, "**", ifelse(utest_res$p.value < 0.05, "*", "")))
    stars <- ifelse(is.na(utest_res$p.value), "", stars)
    ret[["utest_top"]] <- utest_res$top %>%
      round(2) %>%
      as.character() %>%
      paste0(stars)
    ret[["utest_x_min"]] <- utest_res$x_min %>%
      round(2)
    ret[["utest_x_max"]] <- utest_res$x_max %>%
      round(2)
  }
  ret
}

glance_custom.lm <- function(model, ...) {
  ret <- data.frame(dummy = NA)
  Xs <- (model %>% formula() %>% as.character())[[3]]
  if (str_detect(Xs, "I\\(.+\\^2\\)")) {
    reg.variables <- (formula(model) %>% as.character())[[3]] %>% str_split("\\s*\\+\\s*")
    utest_res <- utest(model, reg.variables[[1]][1], reg.variables[[1]][2])
    stars <- ifelse(utest_res$p.value < 0.001, "***", ifelse(utest_res$p.value <
      0.01, "**", ifelse(utest_res$p.value < 0.05, "*", "")))
    stars <- ifelse(is.na(utest_res$p.value), "", stars)
    ret[["utest_top"]] <- utest_res$top %>% round(4) %>% as.character() %>% paste0(stars)
    ret[["utest_x_min"]] <- utest_res$x_min
    ret[["utest_x_max"]] <- utest_res$x_max
  }
  ret
}

library(ggplot2)
library(ggeffects)
flatui <- c("#DF7A5E", "#F0C986", "#82B29A", "#3C405B")

library(reticulate)
use_condaenv("new-africa")

sns <- import("seaborn")
plt <- import("matplotlib.pyplot")
flatui <- c("#DF7A5E", "#F0C986", "#82B29A", "#3C405B")
common_tick_params <- function (ax) {
    ax$tick_params(
        direction = "in",
        length=4,
        width=1.5,
        colors="black",
        grid_color="black",
        grid_alpha=0.5,
    )
}
sns$set_palette(flatui)

reg <- plm(lnco2 ~ lnFDIstock + economic + WGI + natural + military + urban,
           data = data , model = "within", effect = "twoways")

ax <- sns$scatterplot(data = data, x = "lnFDIstock", y = "lnco2", cmap = "mako")

data$military <- mean(data$military)
data$economic <- mean(data$economic)
data$WGI <- mean(data$WGI)
data$natural <- mean(data$natural)
data$urban <- mean(data$urban)


if (class(reg)[1] == 'plm') {
    fef <- fixef(reg, effect = reg$args$effect)
    y <- predict(reg, data) - fef + mean(fef)
} else {
    y <- predict(reg, data)
}

sns$lineplot(x = data[["lnFDIstock"]], y = y, color = flatui[3], ax = ax)
ax %>% common_tick_params()
plt$show()

In [3]:
###产业结构效应（就业）
df <- pdata.frame(read.csv(file.path(path.expand("~"), "OneDrive/PhD Dissertation/Data_Code/Data/index_data_regvariables.csv")),
    index = c("Numeric", "Year")
)
df["SDI_100"] <- df["SDI"] * 100

df_indu <- pdata.frame(read.csv(file.path(path.expand("~"), "OneDrive/PhD Dissertation/Data_Code/Data/industrydata.csv")),
    index = c("Numeric", "Year")
)
df_indu ["SDI_100"] <- df_indu ["SDI"] * 100

df_indu_2 <- pdata.frame(read.csv(file.path(path.expand("~"), "OneDrive/PhD Dissertation/Data_Code/Data/industrydata_2.csv")),
    index = c("Numeric", "Year")
)

df_indu_2["SDI_100"] <- df_indu_2["SDI"] * 100

indu <- list(
    "fixed.individual.square" <- plm(SDI_100 ~ lnFDIstock + I(lnFDIstock^2) + economic + WGI + natural + military + urban,
    data = df, model = "within", effect = "individual"
  ),
   "indu" <- plm(indu_employ ~ lnFDIstock + military + WGI + natural + urban + economic,
    data = df_indu, model = "within", effect = "individual"
  ),
    "indu" <- plm(indu_employ ~ lnFDIstock + I(lnFDIstock^2) + military + WGI + natural + urban + economic,
    data = df_indu, model = "within", effect = "individual"
  ),
   "serv" <- plm(serv_employ ~ lnFDIstock + military + WGI + natural + urban + economic,
    data = df_indu, model = "within", effect = "individual"
  ),
   "serv" <- plm(serv_employ ~ lnFDIstock + I(lnFDIstock^2) + military + WGI + natural + urban + economic,
    data = df_indu, model = "within", effect = "individual"
  ),
   "environ" <- plm(lnenviron ~ lnFDIstock + military + WGI + natural + urban + economic,
    data = df_indu_2, model = "within", effect = "individual"
  ),
   "environ" <- plm(lnenviron ~ lnFDIstock + I(lnFDIstock^2) + military + WGI + natural + urban + economic,
    data = df_indu_2, model = "within", effect = "individual"
  )
)
rows <- tribble(
  ~term, ~indu, ~indu, ~indu, ~serv, ~serv,~serv, ~serv,
  "","可持续发展", "工业","","服务业", "","环境友好型产品出口",""
)
attr(rows, "position") <- c(1)

indu <- modelsummary(
  indu,
  fmt = "%.2f",
  stars = SIG,
  estimate = "{estimate}{stars}",
  statistics = "std.error",
  title = "",
  gof_map = gm,
  coef_map = names,
  add_rows = rows,
  output = "huxtable"
)

indu <- indu %>%
  set_align(everywhere, everywhere, "center") %>%
  set_top_border(row = 1, col = everywhere) %>%
  set_top_border(row = -1, col = everywhere, 0) %>%
  set_bottom_border(row = 2, col = everywhere, 0.4)
quick_docx(indu, file = file.path(Path.HOME, "OneDrive/PhD Dissertation/Regression Results/indu_effect.docx"))
indu

,,(1),(2),(3),(4),(5),(6),(7)
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,,(1),(2),(3),(4),(5),(6),(7)
1.1,,可持续发展,工业,,服务业,,环境友好型产品出口,
2,lnFDIstock,-0.48*,0.67***,1.25***,3.09***,-0.66,0.71***,0.18
3,,(0.28),(0.08),(0.37),(0.15),(0.65),(0.07),(0.38)
4,lnFDIstock2,0.12***,,-0.04,,0.27***,,0.04
5,,(0.02),,(0.03),,(0.05),,(0.03)
6,WGI,-0.43,2.94***,2.89***,4.91***,5.25***,1.12**,1.23***
7,,(0.45),(0.59),(0.59),(1.04),(1.01),(0.44),(0.45)
8,economic,9.49**,-1.23,-1.98,-18.58*,-13.74,3.20,3.47


In [4]:
###技术效应
df_tech <- pdata.frame(read.csv(file.path(path.expand("~"), "OneDrive/PhD Dissertation/Data_Code/Data/technologydata.csv")),
    index = c("Numeric", "Year")
)


Technology <- list(
    "fixed.individual.square" <- plm(SDI_100 ~ lnFDIstock + I(lnFDIstock^2) + economic + WGI + natural + military + urban,
    data = df, model = "within", effect = "individual"
  ),    
    "skills" <- plm(overallin ~ lnFDIstock + military + WGI + natural + urban + economic,
    data = df_tech, model = "within", effect = "individual"
  ),
     "skills" <- plm(overallin ~ lnFDIstock + I(lnFDIstock^2) + military + WGI + natural + urban + economic,
    data = df_tech, model = "within", effect = "individual"
  ),
    "ict" <- plm(ict ~ lnFDIstock + military + WGI + natural + urban + economic,
    data = df_tech, model = "within", effect = "individual"
  ),
     "ict" <- plm(ict ~ lnFDIstock + I(lnFDIstock^2) + military + WGI + natural + urban + economic,
    data = df_tech, model = "within", effect = "individual"
  ),    
    "skills" <- plm(research ~ lnFDIstock + military + WGI + natural + urban + economic,
    data = df_tech, model = "within", effect = "individual"
  ),
     "skills" <- plm(research ~ lnFDIstock + I(lnFDIstock^2) + military + WGI + natural + urban + economic,
    data = df_tech, model = "within", effect = "individual"
  )
    
)

rows <- tribble(
  ~term, ~lnenviron, ~lnenviron,~lnenviron,~lnenviron,~lnenviron,~lnenviron,~lnenviron,
  "","可持续发展", "技术前沿准备指数", "","ICT","","R&D",""
)
attr(rows, "position") <- c(1,2)

Technology <- modelsummary(
  Technology,
  fmt = "%.2f",
  stars = SIG,
  estimate = "{estimate}{stars}",
  statistics = "std.error",
  title = "",
  gof_map = gm,
  coef_map = names,
  add_rows = rows,
  output = "huxtable"
)
Technology <- Technology %>%
  set_align(everywhere, everywhere, "center") %>%
  set_top_border(row = 1, col = everywhere) %>%
  set_top_border(row = -1, col = everywhere, 0) %>%
  set_bottom_border(row = 2, col = everywhere, 0.4)
quick_docx(Technology, file = file.path(Path.HOME, "OneDrive/PhD Dissertation/Regression Results/Technology.docx"))
Technology

,,(1),(2),(3),(4),(5),(6),(7)
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,,(1),(2),(3),(4),(5),(6),(7)
1.1,,可持续发展,技术前沿准备指数,,ICT,,R&D,
2,lnFDIstock,-0.48*,0.02***,0.03**,0.05***,-0.04,0.04***,-0.04***
3,,(0.28),(0.00),(0.01),(0.01),(0.03),(0.00),(0.01)
4,lnFDIstock2,0.12***,,-0.00,,0.01***,,0.01***
5,,(0.02),,(0.00),,(0.00),,(0.00)
6,WGI,-0.43,0.02*,0.02,0.02,0.04,0.03**,0.04***
7,,(0.45),(0.01),(0.01),(0.04),(0.04),(0.01),(0.01)
8,economic,9.49**,-0.38***,-0.38***,-0.36,-0.38,-0.45***,-0.46***


In [5]:
###制度的调节效应
df_institution <- pdata.frame(read.csv(file.path(path.expand("~"), "OneDrive/PhD Dissertation/Data_Code/Data/institutiondata.csv")),
    index = c("Numeric", "Year")
)

df_institution["SDI_100"] <- df_institution["SDI"] * 100
names <- c(
    "SDI_100" = "SDI",    
    "lnFDIstock" = "lnFDIstock",
    "I(lnFDIstock^2)" = "lnFDIstock2", 

    "property" = "institution",
    "I(lnFDIstock * property)"="lnFDIstock*institution",
    "I(lnFDIstock^2 * property)" = "lnFDIstock2*institution",
    
    "environmenttax" = "institution",
    "I(lnFDIstock * environmenttax)"="lnFDIstock*institution",
    "I(lnFDIstock^2 * environmenttax)" = "lnFDIstock2*institution",

    "economic" = "economic",
    "military" = "military",
    "natural" = "natural",
    "urban"= "urban"
    

)


df_institution2 <- pdata.frame(read.csv(file.path(path.expand("~"), "OneDrive/PhD Dissertation/Data_Code/Data/institutiondata_2.csv")),
    index = c("Numeric", "Year")
)
df_institution2["SDI_100"] <- df_institution2["SDI"] * 100

Institution <- list(
    "property" <- plm(SDI_100 ~ lnFDIstock + property + I(lnFDIstock*property)  + I(lnFDIstock^2) + I(lnFDIstock^2*property) 
                      + military + natural + urban + economic,
    data = df_institution, model = "within", effect = "individual"
  ),  
   "environmenttax" <- plm(SDI_100 ~ lnFDIstock + environmenttax + I(lnFDIstock*environmenttax)  + I(lnFDIstock^2) + I(lnFDIstock^2*environmenttax) 
                           + military + natural + urban + economic,
    data = df_institution2, model = "within", effect = "individual"
  )
)

rows <- tribble(
  ~term, ~WGI.time, ~WGI.twoways, 
  "", "产权","环境税",
)
attr(rows, "position") <- c(1)


Institution <- modelsummary(
  Institution,
  fmt = "%.2f",
  stars = SIG,
  estimate = "{estimate}{stars}",
  statistics = "std.error",
  title = "",
  gof_map = gm,
  coef_map = names,
  add_rows = rows,
  output = "huxtable"
)
Institution <- Institution %>%
  set_align(everywhere, everywhere, "center") %>%
  set_top_border(row = 1, col = everywhere) %>%
  set_top_border(row = -1, col = everywhere, 0) %>%
  set_bottom_border(row = 2, col = everywhere, 0.4)
quick_docx(Institution, file = file.path(Path.HOME, "OneDrive/PhD Dissertation/Regression Results/Institution.docx"))
Institution

,,(1),(2)
,<chr>,<chr>,<chr>
1,,(1),(2)
1.1,,产权,环境税
2,lnFDIstock,1.78***,1.21*
3,,(0.46),(0.62)
4,lnFDIstock2,-0.06*,-0.02
5,,(0.03),(0.04)
6,institution,0.30***,7.69***
7,,(0.06),(2.44)
8,lnFDIstock*institution,-0.09***,-2.11***


In [6]:
reg_property<-plm(SDI_100 ~ lnFDIstock + property + I(lnFDIstock*property)  + I(lnFDIstock^2) + I(lnFDIstock^2*property) 
                  + military + natural + urban + economic,
    data = df_institution, model = "within", effect = "individual"
  )

mydf <- ggpredict(reg_property, terms = c("lnFDIstock [all]", "property [meansd]"))
property<-ggplot(mydf, aes(x = x, y = predicted, colour = group, linetype = group)) +
  theme(
    text = element_text(family='Songti SC',size = 23),
    panel.background = element_rect(fill = "white", colour = "grey50"),
    panel.grid = element_blank(),
    legend.position = c(0.8, 0.1)
  ) +  
  geom_line(size=1.5) +
  scale_linetype_manual(values = c('solid', 'dotted','dashed'), labels = c('低产权水平','平均产权水平', '高产权水平')) +
  scale_color_manual(values = c(flatui[1], flatui[2], flatui[3]), labels = c('低产权水平','平均产权水平', '高产权水平')) + 
  labs(
    y = "SDI",
    x = get_x_title(mydf),
    colour = element_blank(),
    linetype = element_blank()
  )
ggsave(file.path(Path.HOME, "OneDrive/PhD Dissertation/Pictures/产权.png"), property)

Warning message in predict.plm(model, newdata = fitfram, type = pr.type, level = 0, :
“Data supplied in argument 'newdata' is not a pdata.frame; weighted mean of fixed effects as in original model used for prediction, see ?predict.plm.”
Could not compute variance-covariance matrix of predictions. No
  confidence intervals are returned.

Warning message:
“Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.”
Saving 7 x 7 in image


In [7]:
reg_environmenttax<-plm(SDI_100 ~ lnFDIstock + environmenttax + I(lnFDIstock*environmenttax)  + I(lnFDIstock^2) + I(lnFDIstock^2*environmenttax) 
                        + military + natural + urban + economic,
    data = df_institution2, model = "within", effect = "individual"
  )

mydf <- ggpredict(reg_environmenttax, terms = c("lnFDIstock [all]", "environmenttax [meansd]"))
environmenttax<-ggplot(mydf, aes(x = x, y = predicted, colour = group, linetype = group)) +
  theme(
    text = element_text(family='Songti SC',size = 23),
    panel.background = element_rect(fill = "white", colour = "grey50"),
    panel.grid = element_blank(),
    legend.position = c(0.8, 0.1)
  ) +  
  geom_line(size=1.5) +
  scale_linetype_manual(values = c('solid','dotted', 'dashed'), labels = c('低环境税水平','平均环境税水平', '高环境税水平')) +
  scale_color_manual(values = c(flatui[1],flatui[4], flatui[3]), labels = c('低环境税水平','平均环境税水平', '高环境税水平')) + 
  labs(
    y = "SDI",
    x = get_x_title(mydf),
    colour = element_blank(),
    linetype = element_blank()
  )
ggsave(file.path(Path.HOME, "OneDrive/PhD Dissertation/Pictures/环境税.png"), environmenttax)

Warning message in predict.plm(model, newdata = fitfram, type = pr.type, level = 0, :
“Data supplied in argument 'newdata' is not a pdata.frame; weighted mean of fixed effects as in original model used for prediction, see ?predict.plm.”
Could not compute variance-covariance matrix of predictions. No
  confidence intervals are returned.

Saving 7 x 7 in image


In [8]:
###金融的调节效应
df_finance <- pdata.frame(read.csv(file.path(path.expand("~"), "OneDrive/PhD Dissertation/Data_Code/Data/financedata.csv")),
    index = c("Numeric", "Year")
)

df_finance_2 <- pdata.frame(read.csv(file.path(path.expand("~"), "OneDrive/PhD Dissertation/Data_Code/Data/financedata_2.csv")),
    index = c("Numeric", "Year")
)

names <- c(
    "SDI" = "SDI",    
    "lnFDIstock" = "lnFDIstock",
    "I(lnFDIstock^2)" = "lnFDIstock2",
    "finance" = "finance",
    "di02" = "finance",
    "I(lnFDIstock * finance)"="lnFDIstock*finance",
    "I(lnFDIstock^2 * finance)" = "lnFDIstock2*finance",
    

    "I(lnFDIstock * di02)"="lnFDIstock*finance",
    "I(lnFDIstock^2 * di02)" = "lnFDIstock2*finance",

    "WGI" = "WGI",    
    "economic" = "economic", 
    "natural" = "natural",
    "urban" = "urban",
    "military" = "military"
)

# df_finance["lnFDIstock_winsor5"] <- as.vector(winsor(df_finance["lnFDIstock"], trim = .05)) 

# df_finance["lnFDIstock_scale"] <- as.vector(scale(df_finance["lnFDIstock"]))
# df_finance["military_scale"] <- as.vector(scale(df_finance["military"]))
# df_finance["military1_scale"] <- as.vector(scale(df_finance["military1"]))
# df_finance["military2_scale"] <- as.vector(scale(df_finance["military2"]))
# df_finance["military3_scale"] <- as.vector(scale(df_finance["military3"]))


df_finance["SDI_100"] <- df_finance["SDI"] * 100
df_finance_2["SDI_100"] <- df_finance_2["SDI"] * 100

finance <- list(
    "finance.twoways" <- plm(SDI_100 ~ lnFDIstock + finance + I(lnFDIstock^2) + I(lnFDIstock*finance) + I(lnFDIstock^2*finance)
                                   + economic + WGI + urban + natural + military,
    data = df_finance, model = "within", effect = "individual"
    ),
    "finance.twoways" <- plm(SDI_100 ~ lnFDIstock + di02 + I(lnFDIstock^2) + I(lnFDIstock*di02) + I(lnFDIstock^2*di02) 
                             + economic + WGI + urban + natural + military,
    data = df_finance_2, model = "within", effect = "individual"
    )
)

rows <- tribble(
  ~term, ~finance.ols, ~finance.time,  
  "", "金融广度", "金融深度"
)
attr(rows, "position") <- c(0)


finance <- modelsummary(
  finance,
  fmt = "%.2f",
  stars = SIG,
  estimate = "{estimate}{stars}",
  statistics = "std.error",
  title = "",
  gof_map = gm,
  coef_map = names,
  output = "huxtable",
  add_rows = rows
)
finance <- finance %>%
  set_align(everywhere, everywhere, "center") %>%
  set_top_border(row = 1, col = everywhere) %>%
  set_top_border(row = -1, col = everywhere, 0) %>%
  set_bottom_border(row = 2, col = everywhere, 0.4)

quick_docx(finance, file = file.path(Path.HOME, "OneDrive/PhD Dissertation/Regression Results/finance.docx"))

finance

,,(1),(2)
,<chr>,<chr>,<chr>
1,,(1),(2)
1.1,,金融广度,金融深度
2,lnFDIstock,0.41,0.30
3,,(0.28),(0.31)
4,lnFDIstock2,0.03,0.04
5,,(0.02),(0.02)
6,finance,1.52***,0.53***
7,,(0.27),(0.07)
8,lnFDIstock*finance,-0.39***,-0.10***


In [9]:
finance1<-plm(SDI_100 ~ lnFDIstock + finance + I(lnFDIstock^2) + I(lnFDIstock*finance) + I(lnFDIstock^2*finance) 
              + economic + WGI + urban + natural + military ,
    data = df_finance, model = "within", effect = "individual"
    )

mydf <- ggpredict(finance1, terms = c("lnFDIstock [all]", "finance [meansd]"))
finance1<-ggplot(mydf, aes(x = x, y = predicted, colour = group, linetype = group)) +
  theme(
    text = element_text(family='Songti SC',size = 23),
    panel.background = element_rect(fill = "white", colour = "grey50"),
    panel.grid = element_blank(),
    legend.position = c(0.75, 0.1)
  ) +  
  geom_line(size=1.5) +
  scale_linetype_manual(values = c('solid','dotted', 'dashed'), labels = c('低金融广度水平','平均金融广度水平', '高金融广度水平')) +
  scale_color_manual(values = c(flatui[1],flatui[4], flatui[3]), labels = c('低金融广度水平','平均金融广度水平', '高金融广度水平')) + 
  labs(
    y = "SDI",
    x = get_x_title(mydf),
    colour = element_blank(),
    linetype = element_blank()
  )
ggsave(file.path(Path.HOME, "OneDrive/PhD Dissertation/Pictures/金融广度.png"), finance1)

Warning message in predict.plm(model, newdata = fitfram, type = pr.type, level = 0, :
“Data supplied in argument 'newdata' is not a pdata.frame; weighted mean of fixed effects as in original model used for prediction, see ?predict.plm.”
Could not compute variance-covariance matrix of predictions. No
  confidence intervals are returned.

Saving 7 x 7 in image


In [10]:
finance2<-plm(SDI_100 ~ lnFDIstock + di02 + I(lnFDIstock^2) + I(lnFDIstock*di02) + I(lnFDIstock^2*di02) 
              + economic + WGI + urban + natural + military,
    data = df_finance_2, model = "within", effect = "individual"
    )

mydf <- ggpredict(finance2, terms = c("lnFDIstock [all]", "di02 [meansd]"))
finance2<-ggplot(mydf, aes(x = x, y = predicted, colour = group, linetype = group)) +
  theme(
    text = element_text(family='Songti SC',size = 23),
    panel.background = element_rect(fill = "white", colour = "grey50"),
    panel.grid = element_blank(),
    legend.position = c(0.75, 0.7)
  ) +  
  geom_line(size=1.5) +
  scale_linetype_manual(values = c('solid','dotted', 'dashed'), labels = c('低金融深度水平','平均金融深度水平', '高金融深度水平')) +
  scale_color_manual(values = c(flatui[1],flatui[4], flatui[3]), labels = c('低金融深度水平','平均金融深度水平', '高金融深度水平')) + 
  labs(
    y = "SDI",
    x = get_x_title(mydf),
    colour = element_blank(),
    linetype = element_blank()
  )
ggsave(file.path(Path.HOME, "OneDrive/PhD Dissertation/Pictures/金融深度.png"), finance2)

Warning message in predict.plm(model, newdata = fitfram, type = pr.type, level = 0, :
“Data supplied in argument 'newdata' is not a pdata.frame; weighted mean of fixed effects as in original model used for prediction, see ?predict.plm.”
Could not compute variance-covariance matrix of predictions. No
  confidence intervals are returned.

Saving 7 x 7 in image
